<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/11_document_clustering_%E6%8E%88%E6%A5%AD%E4%B8%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング
* クラスタリングの代表的な手法であるk平均法を使ってみる。
* ついでに、言語モデルを使ったテキストマイニングを体験してみる。

## 例題: 文書クラスタリング

* Transformerベースの日本語対応言語モデルを使って、テキストのベクトル表現を得る。
  * Transformerというニューラルネットワークについては、いずれ学びます。
  * 有名な解説記事 https://jalammar.github.io/illustrated-transformer/
* テキストをベクトルとして表現することを「embedする」と言う。
  * embedすることで得られるベクトルのことを「embedding」と言う。
* そして、テキストのembeddingをk平均法でクラスタリングする。

* ランタイムのタイプをGPUにしておく。

## インストール

### spaCyの日本語モデル

* 日本語テキストを形態素解析するために使う。
  * たぶん、セッションの再起動」は不要。

In [1]:
!python -m spacy download ja_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Hugging Faceのdatasetsライブラリ

* ライブドアニュースコーパスを取得するために使う。

In [2]:
!pip install --upgrade datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.0
    Uninstalling huggingface-hub-0.33.0:
      Successfully uninstalled huggingface-hub-0.33.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_s

### SentenceTransformersライブラリ
* 言語モデルを使ってテキストを埋め込む際に便利なライブラリ。
  * https://sbert.net/index.html

In [3]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

## インポート

In [4]:
from tqdm.auto import tqdm
import collections
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy

from datasets import load_dataset
from transformers import set_seed
from sentence_transformers import SentenceTransformer

# 再現性の確保
set_seed(1234)

## データセット
* livedoorニュースコーパスを使う。

In [5]:
dataset = load_dataset(
  "shunk031/livedoor-news-corpus",
  train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
  random_state=42,
  shuffle=True,
  trust_remote_code=True,
)

num_categories = len(set(dataset["train"]["category"]))

category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

print(f"num_categories: {num_categories}")
print(f"category_names: {category_names}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

livedoor-news-corpus.py: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

num_categories: 9
category_names: ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']


In [6]:
dataset["train"][0]

{'url': 'http://news.livedoor.com/article/detail/5834377/',
 'date': '2011-09-04T08:30:00+0900',
 'title': '【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露',
 'content': '10月、東京で行われる世界体操では、兄・田中和仁＆弟・田中佑典とともに3兄弟で出場を決めた田中理恵。2日、フジテレビ「すぽると！」では「田中3兄弟SP対談」と題し、兄弟3人によるトークの模様が放送された。  「お兄ちゃんは、形にはまって美しい体操をしている感じがする。教科書に載るような。佑典は綺麗で、かつオシャレやなって一言言いたくなる演技」、その他にも、「ずっと（兄弟を）見ていたから、他の女子の“こういう選手になりたい”というのがない。弟の鉄棒みたいなオシャレ演技したいし、お兄ちゃんみたいな綺麗な線出したいしという気持ちが強い」と語った理恵。  対して、理恵が“美人アスリート”と呼ばれることについて、和仁は「妹が出てるってことで、兄としては嬉しいけど、色んな人に言われても“ふーん”っていって終わり」と素っ気なく、佑典は「美人アスリート。まあ、不細工アスリートよりは、美人アスリートなんじゃないですかね」と語る。  また、理恵の学生時代の様子について、「和歌山の（実家の）時は部屋汚かったです」と明かす和仁。これには理恵も「ここで言う？」と呆れたが、和仁は、お構いなしに「高校の時、髪染めたな。“染めてない”って言ってたけど、染めてたな。で、（父・章二さんから）リモコン飛んできたんだよな。おとんが教師なのに、そこの生徒でよう染めたなっていうのは、皆思ってたよ」と暴露を続けた。  ・田中理恵 写真ギャラリー',
 'category': 6}

In [8]:
dataset["train"]["title"][:10]

['【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露',
 '「美人で何が悪い！？」 負け美女・小島慶子×犬山紙子対談 3/3',
 'もうひとつのアカデミー賞!?\u3000スカパーアダルト放送大賞が決定！',
 '渡邉美樹氏「猫ひろしさんのカンボジア国籍取得 納得いきません」',
 'やくみつるさんの「DeNAが許せない」発言に批判殺到',
 'ポスターのテーマは、剣心の“封印された狂気”',
 '悩ましき女友だちとの格差問題',
 'ラジオ体操は「究極のエクササイズ」ってホント？',
 'オトナ女子たちの圧倒的支持をうけ、ドラマ10『はつ恋』の一挙再放送が決定！',
 '映画『サルベージ・マイス』主題歌に、ももいろクローバーＺの新曲が決定']

In [9]:
dataset["train"]["content"][0]

'10月、東京で行われる世界体操では、兄・田中和仁＆弟・田中佑典とともに3兄弟で出場を決めた田中理恵。2日、フジテレビ「すぽると！」では「田中3兄弟SP対談」と題し、兄弟3人によるトークの模様が放送された。  「お兄ちゃんは、形にはまって美しい体操をしている感じがする。教科書に載るような。佑典は綺麗で、かつオシャレやなって一言言いたくなる演技」、その他にも、「ずっと（兄弟を）見ていたから、他の女子の“こういう選手になりたい”というのがない。弟の鉄棒みたいなオシャレ演技したいし、お兄ちゃんみたいな綺麗な線出したいしという気持ちが強い」と語った理恵。  対して、理恵が“美人アスリート”と呼ばれることについて、和仁は「妹が出てるってことで、兄としては嬉しいけど、色んな人に言われても“ふーん”っていって終わり」と素っ気なく、佑典は「美人アスリート。まあ、不細工アスリートよりは、美人アスリートなんじゃないですかね」と語る。  また、理恵の学生時代の様子について、「和歌山の（実家の）時は部屋汚かったです」と明かす和仁。これには理恵も「ここで言う？」と呆れたが、和仁は、お構いなしに「高校の時、髪染めたな。“染めてない”って言ってたけど、染めてたな。で、（父・章二さんから）リモコン飛んできたんだよな。おとんが教師なのに、そこの生徒でよう染めたなっていうのは、皆思ってたよ」と暴露を続けた。  ・田中理恵 写真ギャラリー'

## 多言語E5による埋め込み

* Multilingual E5を使う。
  * テキストのembeddingにおいて優れている言語モデル。
  * 論文 https://arxiv.org/abs/2402.05672
  * Hugging Face https://huggingface.co/intfloat/multilingual-e5-large-instruct

* 参考: テキスト埋め込みのleaderboard
  * https://huggingface.co/spaces/mteb/leaderboard

* SentenceTransformerを使ったテキストの埋め込みについては、下のWebページを参照。
  * https://sbert.net/examples/sentence_transformer/applications/computing-embeddings/README.html

In [10]:
model_id = "intfloat/multilingual-e5-large-instruct"
model = SentenceTransformer(model_id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

* 試しに、一つだけ、テキストを埋め込んでみる。

In [11]:
dataset["train"][0]["title"]

'【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露'

In [12]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [16]:
model.encode(dataset["train"][0]["title"])

array([ 0.01802401,  0.03642237, -0.00951136, ..., -0.03236501,
       -0.03817462,  0.01583127], dtype=float32)

* ライブドアニュースコーパスの全タイトルを埋め込む。

In [17]:
embeddings = model.encode(dataset["train"]["title"], show_progress_bar=True)

Batches:   0%|          | 0/185 [00:00<?, ?it/s]

* 埋め込みは普通にNumPyの配列として得られている。

In [18]:
type(embeddings)

numpy.ndarray

In [19]:
embeddings.shape

(5894, 1024)

* 全記事内容を埋め込むには以下のようにする。  
  * RTX3080搭載PCを使うと1分で終わる。

In [ ]:
#content_embeddings = model.encode(dataset["train"]["content"], show_progress_bar=True)

* ただし、どのテキストも先頭から512トークンで切られていることに注意。
  * 長いテキストは、途中までの内容しかembeddingに反映されない。
  * それでも、分類やクラスタリングがうまくいくことも多い。

In [20]:
model.max_seq_length

512

* トークン数の調べ方
  * トークナイザにテキストを分割させる。
  * 分割によって得られたトークンの個数を数える。

In [22]:
dataset["train"][0]["title"]

'【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露'

In [21]:
model.tokenize([dataset["train"][0]["title"]])

{'input_ids': tensor([[     0,   5946,  43488,      7,  20413,   2728,   4742,  40019,   1925,
            6676,    514,   6986, 126235,     37,  67156,    281,   7252, 162829,
           11264,  36948, 124408,   7116,    251, 174856,      2]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1]])}

In [23]:
(model.tokenize([dataset["train"][0]["title"]])['input_ids']).shape[1]

25

* 埋め込みを保存。

In [24]:
with open('embeddings.npy', 'wb') as f:
  np.save(f, embeddings)

In [ ]:
#with open('content_embeddings.npy', 'wb') as f:
#  np.save(f, content_embeddings)

* 読み込みは以下のようにする。

In [25]:
with open('embeddings.npy', 'rb') as f:
  embeddings = np.load(f)

In [ ]:
#with open('content_embeddings.npy', 'rb') as f:
#  content_embeddings = np.load(f)

## クラスタのラベリングに使う単語の抽出

* 全テキストを形態素解析する。
  * 形態素解析＝単語への分割

In [26]:
nlp = spacy.load("ja_core_news_sm")
corpus = []
for text in tqdm(dataset["train"]["title"]):
  corpus.append(" ".join([token.lemma_ for token in nlp(text)]))

  0%|          | 0/5894 [00:00<?, ?it/s]

In [28]:
dataset["train"][0]["title"]

'【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露'

In [32]:
corpus[0]

'【 SPORTS watch 】 体操 ・ 田中 理恵 、 兄 が 学生 時代 の エピソード を 暴露'

* scikit-learnでTF-IDFを計算する。
* `TfidfVectorizer`の`min_df`パラメータは適当に調節する。
  * クラスタのラベリングに向かないマイナーな単語が含まれないようにする。

In [30]:
vectorizer = TfidfVectorizer(min_df=20)

In [31]:
X_train = vectorizer.fit_transform(corpus)

In [35]:
X_train = X_train.toarray()

In [41]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
X_train.shape

(5894, 447)

In [38]:
vocab = np.array(vectorizer.get_feature_names_out())

In [39]:
vocab.size

447

In [47]:
print(list(vocab))

['01', '02', '04', '05', '06', '07', '10', '100', '1000万', '11', '12', '13', '15', '1日', '20', '2011', '2012', '23', '24', '30', '40', '48', 'akb', 'android', 'aquos', 'arrows', 'au', 'by', 'cafe', 'cm', 'cpu', 'dvd', 'facebook', 'fi', 'galaxy', 'google', 'hd', 'ics', 'ipad', 'iphone', 'is', 'isw', 'it', 'kddi', 'lte', 'mac', 'medias', 'nhk', 'note', 'ntt', 'optimus', 'os', 'pc', 'phone', 'presented', 'salon', 'sc', 'sh', 'sports', 'twitter', 'vol', 'vs', 'watch', 'wi', 'wimax', 'windows', 'xi', 'xperia', 'あなた', 'あの', 'ある', 'いい', 'いう', 'いく', 'いる', 'おく', 'お気に入り', 'から', 'かわいい', 'くる', 'くれる', 'こと', 'この', 'これ', 'さん', 'しまう', 'すぎる', 'する', 'せる', 'そう', 'その', 'たい', 'たち', 'ため', 'だけ', 'ちゃう', 'ちゃん', 'って', 'っと', 'ついに', 'つく', 'てる', 'できる', 'です', 'どう', 'どこ', 'ない', 'なし', 'なぜ', 'なでしこ', 'など', 'なる', 'なん', 'にて', 'にゅう', 'べし', 'ます', 'まで', 'まとめ', 'みる', 'みんな', 'もう', 'もの', 'やすい', 'やる', 'ゆるい', 'よう', 'より', 'よる', 'られる', 'れる', 'わかる', 'アイテム', 'アップ', 'アップル', 'アナ', 'アプリ', 'イケショップ', 'イベント', 'インタビュー', 'インチ', 'オススメ', 'オトナ

## ラベリング用単語の埋め込み

* 各単語について、その単語を含むテキストの埋め込みベクトルの加重平均を求める。
* 加重平均の重みは、各テキストにおけるその単語のTF-IDFの値を使って定める。

In [44]:
X_train.sum(0)

array([ 14.23737419,   8.29165953,   8.89160565,  10.98194494,
        18.93665008,   9.46061414,  40.07367426,  14.28822988,
        14.41911992,  22.80643974,  18.05936778,  11.4279328 ,
        11.59444793,  11.84556731,  20.7174215 ,  23.58182266,
        41.04647437,  12.39391169,  10.63096185,  17.4181583 ,
        10.99017477,  27.09842548,  35.27405367,  78.35690806,
        15.26955653,  10.83606064,  24.80076668,  23.88017003,
        20.80809394,  14.17248708,  10.96561034,  20.0860085 ,
        18.03084136,  14.36513671,  24.92943815,  22.90955665,
        10.09431811,  12.73046169,  24.04995493,  76.33793521,
         7.58491082,   8.07353078,  15.39973941,  16.2993322 ,
        12.22679908,  10.5302457 ,   9.04033087,  12.29946058,
         8.40654504,  43.04142467,   8.86133668,   9.48350301,
        23.8057352 ,  21.52179432,  19.95833812,  29.16454774,
        13.14260003,  18.92114609, 154.76217334,  14.14442497,
        72.94053245,  12.27233884, 154.76217334,  14.92

In [45]:
text_weights = X_train / X_train.sum(0)

In [48]:
vocab_embeddings = np.dot(text_weights.T, embeddings)

## 文書クラスタリング



In [49]:
embeddings.shape

(5894, 1024)

* k-meansのしくみ

* 初期化

In [64]:
n_clusters = 20
assignments = np.random.randint(0, n_clusters, X_train.shape[0])

In [70]:
assignments

array([17,  2, 15, ..., 11, 16,  3])

* クラスタの重心の計算

In [75]:
mean_vectors = []
for k in range(n_clusters):
  mean_vectors.append(embeddings[assignments == k].mean(0))

In [77]:
mean_vectors = np.array(mean_vectors)

In [80]:
mean_vectors

array([[ 0.01707397,  0.02981332, -0.01970446, ..., -0.01612372,
        -0.02381798,  0.00840845],
       [ 0.0182366 ,  0.02931906, -0.02000081, ..., -0.01880646,
        -0.02486639,  0.00933847],
       [ 0.01747563,  0.02949889, -0.01920002, ..., -0.01904421,
        -0.02630495,  0.00825417],
       ...,
       [ 0.01766788,  0.02866561, -0.01950578, ..., -0.0175311 ,
        -0.02687437,  0.0075715 ],
       [ 0.0179808 ,  0.02798694, -0.0185404 , ..., -0.01901085,
        -0.02522602,  0.00793715],
       [ 0.01772811,  0.02894015, -0.01936278, ..., -0.01703352,
        -0.0260472 ,  0.00822714]], dtype=float32)

* 各ベクトルに最も近い重心ベクトルを見つけて、クラスタを割り当て直す

In [81]:
np.sqrt(((embeddings[0] - mean_vectors[0]) ** 2).sum())

np.float32(0.41155395)

In [83]:
distances = []
for k in range(n_clusters):
  distances.append(np.linalg.norm(embeddings[0] - mean_vectors[k]))
distances

[np.float32(0.41155395),
 np.float32(0.40537438),
 np.float32(0.40704802),
 np.float32(0.41057798),
 np.float32(0.4071495),
 np.float32(0.4127908),
 np.float32(0.40746143),
 np.float32(0.4086204),
 np.float32(0.4083532),
 np.float32(0.40689692),
 np.float32(0.40787512),
 np.float32(0.40736282),
 np.float32(0.40928462),
 np.float32(0.41039932),
 np.float32(0.4102789),
 np.float32(0.41333348),
 np.float32(0.41303244),
 np.float32(0.4075859),
 np.float32(0.4041526),
 np.float32(0.40716204)]

### k-平均法によるクラスタリング

In [84]:
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings)
#kmeans.fit(content_embeddings) # 本文の場合はこちら。
centers = kmeans.cluster_centers_

In [94]:
centers

array([[ 0.01430322,  0.03136253, -0.02514517, ..., -0.02268711,
        -0.0266082 ,  0.01325098],
       [ 0.0189735 ,  0.02687819, -0.02668014, ..., -0.01993991,
        -0.01975193,  0.01015017],
       [ 0.01751033,  0.02260082, -0.02669384, ..., -0.01917516,
        -0.02717998,  0.01307425],
       ...,
       [ 0.02170468,  0.02648552, -0.02336103, ..., -0.01304382,
        -0.0181316 ,  0.00705924],
       [ 0.01176104,  0.02763935, -0.02222423, ..., -0.02498448,
        -0.0299366 ,  0.01751282],
       [ 0.02082054,  0.0304917 , -0.02854665, ..., -0.02131261,
        -0.02837157,  0.01084703]], dtype=float32)

* クラスタの重心を保存。

In [85]:
with open(f'centers_{n_clusters}.npy', 'wb') as f:
  np.save(f, centers)

In [86]:
with open(f'centers_{n_clusters}.npy', 'rb') as f:
  centers = np.load(f)

### クラスタのサイズを調べる

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [87]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [88]:
print([sorted(size_dict.items(), key=lambda item: item[1], reverse=True)])

[[(np.int32(4), np.int64(498)), (np.int32(13), np.int64(450)), (np.int32(8), np.int64(388)), (np.int32(3), np.int64(360)), (np.int32(14), np.int64(346)), (np.int32(5), np.int64(343)), (np.int32(19), np.int64(343)), (np.int32(9), np.int64(341)), (np.int32(10), np.int64(320)), (np.int32(11), np.int64(290)), (np.int32(7), np.int64(287)), (np.int32(0), np.int64(280)), (np.int32(1), np.int64(280)), (np.int32(15), np.int64(273)), (np.int32(17), np.int64(236)), (np.int32(2), np.int64(211)), (np.int32(16), np.int64(197)), (np.int32(6), np.int64(180)), (np.int32(18), np.int64(154)), (np.int32(12), np.int64(117))]]


## クラスタのラベリング
* 各クラスタの重心に近い単語でラベリングする。

* テキストの埋め込みは、長さ1のベクトルになっている。

In [89]:
np.linalg.norm(embeddings, axis=-1)

array([1.        , 1.        , 0.99999994, ..., 1.        , 1.        ,
       1.        ], dtype=float32)

* テキストとラベリング用の単語との類似度はコサイン類似度で測る。

In [92]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vocab_embeddings, centers)

In [96]:
vocab_embeddings.shape

(447, 1024)

In [97]:
centers.shape

(20, 1024)

In [95]:
similarities.shape

(447, 20)

* 重心に近い順に30個の単語を表示する。

In [93]:
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])
  print(vocab[indices[:30]])

['どう' 'って' 'あなた' 'いる' 'なぜ' '本当' '事情' '男性' '好き' 'なる' 'ある' '女性' '思う' 'しまう'
 'たい' 'どこ' 'もの' 'その' 'こと' '聞く' 'たち' 'れる' '自分' 'べし' '必要' 'いう' 'ない' '女子'
 '彼女' '合う']
['デジ' '使える' '使う' '機能' '携帯' 'スマホ' 'ipad' '利用' 'iphone' '可能' 'できる' '端末'
 'アップル' '情報' '高い' '専用' '話題' 'みる' 'サービス' '発表' 'ユーザー' '新しい' 'から' 'ニュース' '開発'
 '紹介' 'it' 'わかる' '登場' 'おく']
['スマートフォン' 'レポート' 'スマホ' '対応' '06' '05' 'galaxy' 'コンパクト' '試す' '搭載' '01'
 'ドコモ' 'モバイル' '入り' '画面' '向け' '全部' 'レビュー' 'optimus' '防水' '04' '機能' '動画'
 'phone' 'sh' 'モデル' '通信' '07' 'sc' '発売']
['恋愛' 'しまう' '女子' 'たち' 'たい' '事情' '独女' '男性' '女性' '合う' 'いる' '好き' '彼女' 'なる'
 '思う' 'モテる' 'こと' 'れる' '自分' '悩み' 'られる' '聞く' 'せる' '本当' '結婚' '見る' 'って' 'vol'
 '男子' '幸せ']
['映画' '公開' '映像' '解禁' '作品' '特別' 'まとめ' '最高' '予告' 'スター' '決定' '編集' '最強' '誕生'
 '読み' '世界' '上陸' '週末' 'dvd' '挑戦' '込む' 'ドラマ' '国際' 'れる' 'する' '注目' '少女' 'から'
 'られる' '批評']
['さん' 'ちゃん' '人気' 'れる' 'する' 'くる' 'すぎる' 'いい' 'なる' 'あの' 'から' 'いく' 'いる' '見る'
 'られる' '込む' 'たい' 'もう' 'てる' 'せる' 'ちゃう' '出演' 'ある' '出す' 'ない' 'いう' '出る' 'ます'
 '注目' 'この']
['仕事' '会社' 'v

# プランナー課題１１
* それぞれのクラスタについて、重心に近い元々のテキスト（つまり記事タイトル）を5件ずつ表示させてみよう。
* それらのテキストの内容に、上で得たラベルが合っているかどうか、確かめよう。